# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [8]:
import pymysql
from sqlalchemy import create_engine

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [13]:
driver = 'mysql+pymysql:'
user = 'guest'
password = 'relational'
ip = 'relational.fit.cvut.cz'
database= 'stats'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


#### 4. Import the users table.

In [30]:
query = "SELECT * FROM users"
users = pd.read_sql(query, engine)

#### 5. Rename Id column to userId.

In [40]:
users.columns = ['UserId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl']
users.head()

,UserId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 6. Import the posts table. 

In [34]:
query = "SELECT * FROM posts"
posts = pd.read_sql(query, engine)

#### 7. Rename Id column to postId and OwnerUserId to userId.

In [49]:
posts.columns = ['OwnerUserid', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName']

#### 8. Define new dataframes for users and posts with the following selected columns:
**users columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts columns**: postId, Score, userID, ViewCount, CommentCount

In [50]:
user1 = users[['UserId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
post1 = posts[['PostTypeId', 'Score', 'OwnerUserId', 'ViewCount', 'CommentCount']]

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [54]:
merged = users.merge(posts, left_on= 'UserId', right_on= 'OwnerUserId')

#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [55]:
merged.isnull().sum()

UserId                       0
Reputation                   0
CreationDate                 0
DisplayName                  0
LastAccessDate               0
WebsiteUrl               58513
Location                 50132
AboutMe                  48900
Views                        0
UpVotes                      0
DownVotes                    0
AccountId                    0
Age                      63780
ProfileImageUrl          66521
OwnerUserid                  0
PostTypeId                   0
AcceptedAnswerId         76021
CreaionDate                  0
Score                        0
ViewCount                48396
Body                       220
OwnerUserId                  0
LasActivityDate              0
Title                    48396
Tags                     48396
AnswerCount              48396
CommentCount                 0
FavoriteCount            77605
LastEditorUserId         46456
LastEditDate             46228
CommunityOwnedDate       88197
ParentId                 43484
ClosedDa

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [58]:
merged = merged.fillna(0)

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [59]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90584 entries, 0 to 90583
Data columns (total 35 columns):
UserId                   90584 non-null int64
Reputation               90584 non-null int64
CreationDate             90584 non-null datetime64[ns]
DisplayName              90584 non-null object
LastAccessDate           90584 non-null datetime64[ns]
WebsiteUrl               90584 non-null object
Location                 90584 non-null object
AboutMe                  90584 non-null object
Views                    90584 non-null int64
UpVotes                  90584 non-null int64
DownVotes                90584 non-null int64
AccountId                90584 non-null int64
Age                      90584 non-null float64
ProfileImageUrl          90584 non-null object
OwnerUserid              90584 non-null int64
PostTypeId               90584 non-null int64
AcceptedAnswerId         90584 non-null float64
CreaionDate              90584 non-null datetime64[ns]
Score                    90

In [75]:
merged['LastEditDate'] = merged['LastEditDate'][merged['LastEditDate'] != 0]

In [76]:
merged['LastEditDate'] = pd.to_datetime(merged['LastEditDate'])

In [79]:
merged.head()

,UserId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName,LastEditDateError
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,...,0,0.0,-1.0,2014-04-23 13:43:43,0,0.0,0,0,0,2014-04-23 13:43:43
1,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,...,0,0.0,-1.0,2011-03-21 17:40:28,0,0.0,0,0,0,2011-03-21 17:40:28
2,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,...,0,0.0,-1.0,2011-03-21 17:46:43,0,0.0,0,0,0,2011-03-21 17:46:43
3,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,...,0,0.0,919.0,2011-03-30 19:23:14,0,0.0,0,0,0,2011-03-30 19:23:14
4,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,...,0,0.0,919.0,2011-03-30 19:23:14,0,0.0,0,0,0,2011-03-30 19:23:14
